In [1]:
import numpy as np
from scipy import integrate, interpolate

In [2]:
data = np.loadtxt('ВнешнийЦилиндр.0', dtype=np.float64, skiprows=2, usecols=(1, 2, 3, 4, 5, 6))
train_points = data[:, :3]
train_B = data[:, 3:]
data = np.loadtxt('Внутренний Цилиндр.0', dtype=np.float64, skiprows=2, usecols=(1, 2, 3, 4, 5, 6))
test_points = data[:, :3]
test_B = data[:, 3:]

Calc RBF approx. from train points

In [3]:
rbfB = interpolate.RBFInterpolator(train_points, train_B, kernel='linear', neighbors=9)

Geometric algebra $\mathbf B$ approx. is $\mathbf B(r') = \frac{1}{4\pi} \oint\limits_S \frac{r-r'}{|r-r'|^3} n \mathbf B(r) dS$

$\mathbf B(r') = \frac{1}{4\pi} \oint\limits_S \frac{r-r'}{|r-r'|^3} (\mathbf B(r) \cdot n) + \frac{r-r'}{|r-r'|^3} \times (\mathbf B(r) \times n) dS$


In [4]:
def calc_B(r0, R, Zmin, Zmax, _B, epsabs=1e-3, epsrel=1e-4):
    def kernel(r, n):
        b = _B([r])[0]
        dr = r - np.array(r0)
        dr = dr / (4 * np.pi * np.linalg.norm(dr, axis=-1, keepdims=True) ** 3)
        return np.concatenate([dr * np.sum(b * n, axis=-1, keepdims=True) + np.cross(dr, np.cross(b, n)), np.sum(dr * np.cross(b, n), axis=-1, keepdims=True)], axis=-1)
    def internal(phi):
        top, _ = integrate.quad_vec(lambda r: kernel([r * np.cos(phi), r * np.sin(phi), Zmax], [0, 0, 1]) * r, 0, R, epsabs=epsabs, epsrel=epsrel)
        bottom, _ = integrate.quad_vec(lambda r: kernel([r * np.cos(phi), r * np.sin(phi), Zmin], [0, 0, -1]) * r, 0, R, epsabs=epsabs, epsrel=epsrel)
        side, _ = integrate.quad_vec(lambda z: kernel([R * np.cos(phi), R * np.sin(phi), z], [np.cos(phi), np.sin(phi), 0]) * R, Zmin, Zmax, epsabs=epsabs, epsrel=epsrel)
        return top + bottom + side
    res, _ = integrate.quad_vec(internal, 0, 2*np.pi, epsabs=epsabs, epsrel=epsrel)
    print(f'Internal approx. error is {res[3] / np.linalg.norm(res)}')
    return res[:3]

In [5]:
def err(i, _B):
    return np.linalg.norm(_B(test_points[i]) - test_B[i])/np.linalg.norm(test_B[i])
for i in range(0, len(test_points), 200):
    print(f'Error of RBF interpolation of point {test_points[i]} is {err(i, lambda r0: rbfB([r0])[0])}')
    print(f'Error of approx from RBF of point {test_points[i]} is {err(i, lambda r0: calc_B(r0, 1, -1.5, 1.5, rbfB))}')

Error of RBF interpolation of point [ 0.8  0.  -0.8] is 0.03286119410489847
Internal approx. error is -3.0771375832755256e-08
Error of approx from RBF of point [ 0.8  0.  -0.8] is 3.1633557732343294e-05
Error of RBF interpolation of point [4.8985872e-17 8.0000000e-01 4.0000000e-01] is 0.02030303349494282
Internal approx. error is -5.886682441061342e-08
Error of approx from RBF of point [4.8985872e-17 8.0000000e-01 4.0000000e-01] is 2.3691336320788497e-05
Error of RBF interpolation of point [-1.60000000e-01  1.95943488e-17 -1.00000000e+00] is 0.15293411839141235
Internal approx. error is 6.034298090865882e-08
Error of approx from RBF of point [-1.60000000e-01  1.95943488e-17 -1.00000000e+00] is 2.694872260245312e-05
Error of RBF interpolation of point [-7.34788079e-17 -4.00000000e-01 -1.00000000e+00] is 0.16032199941072514
Internal approx. error is 4.084770368599189e-08
Error of approx from RBF of point [-7.34788079e-17 -4.00000000e-01 -1.00000000e+00] is 2.749939807197417e-05
Error of 